In [1]:
# tips: https://stackoverflow.com/questions/23317458/how-to-remove-punctuation
# copy right from TensorFlow team
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
# with customization

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import collections
import math
import os
import sys
import random
from tempfile import gettempdir
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer

In [2]:
url='http://mattmahoney.net/dc/'

def DownloadData(filename):
    path = os.getcwd()
    local_filename = os.path.join(path,filename)
    if not os.path.exists(local_filename):
        local_filename, _ =urllib.request.urlretrieve(url+filename,local_filename)
    statinfo = os.stat(local_filename)
    print('the size of the file', statinfo.st_size)
    return local_filename

filename=DownloadData('en_US.news.txt')

the size of the file 205811889


In [3]:
def readdata(filename):
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

def readdata2(filename, remove_stopwords = True):
    with open(filename,'r+',encoding="utf-8") as f:
        lines = f.readlines()
        data = []
        toker = RegexpTokenizer(r'\w+')
        for line in lines:
            line = line.strip().lower()
            data.extend(toker.tokenize(line))
        if remove_stopwords:
            stop_words = set(stopwords.words('english'))
            data = list(filter(lambda x: x not in stop_words, data))
    return data

volcabulary = readdata2(filename)
print('data size', len(volcabulary))

data size 20548326


In [4]:
print(volcabulary[1:10])
volcabulary_size = 50000
'''
data - list of codes(integers ie, 0 - volcabulary_size-1)
count - map of words(strings) to count the cooccurence
refdictionary - map of words(strings) to their codes(int) -> for word2int
reversed_refdictionary - map codes(int) to the words -> int2word
'''
def builddataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words-1)) #unigram count only
    refdictionary = dict()
    for word, _ in count:
        refdictionary[word]=len(refdictionary)
    data=list()
    unk_count=0
    for word in words:
        index=refdictionary.get(word, 0)
        if index==0:#refdictionary['UNK']
            unk_count+=1
        data.append(index)
    count[0][1]=unk_count
    reversed_refdictionary = dict(zip(refdictionary.values(),refdictionary.keys()))
    return data, count, refdictionary, reversed_refdictionary

data, count, dictionary, reverse_dictionary = builddataset(volcabulary, volcabulary_size)
del volcabulary #clear memory
print('Sample Data', data[:10], [reverse_dictionary[i] for i in data[:10]])

['alone', 'apparently', 'st', 'louis', 'plant', 'close', 'would', 'die', 'old']
Sample Data [36, 1276, 1869, 71, 256, 791, 313, 4, 2657, 58] ['home', 'alone', 'apparently', 'st', 'louis', 'plant', 'close', 'would', 'die', 'old']


In [5]:
'''
batch_size = Batch size
num_skips = How many words to consider left and right
num_skips = How many times to reuse the input to generate the label
num_sampled = Number of negative examples to sample
'''

data_index=0
def gen_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size%num_skips ==0
    assert num_skips <=2*skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size,1), dtype=np.int32)

    span = 2*skip_window+1 # window seen  maximum (case of skip_window=1, span=3)
    buffer1=collections.deque(maxlen=span) #window
    if data_index+span>len(data):
        data_index=0
    buffer1.extend(data[data_index:data_index+span])
    data_index+=span
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w!= skip_window] # not taking the center word always ie, when w=1(considering skip_window=1)
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i*num_skips+j]=buffer1[skip_window] # buffer1[1] --> alwasys fixed
            labels[i*num_skips+j,0]=buffer1[context_word] # buffer1[0] and buffer1[2] both cotext words
        if data_index == len(data):
            buffer1[:]=data[:span]
            data_index=span
        else:
            buffer1.append(data[data_index])
            data_index+=1
    #Backtrack to avoid skipping words in the end of a batch
    data_index = (data_index+len(data)-span)%len(data)
    return batch, labels

# Example is given with batch size = 8 , we can go with bigger batch sizes as well
batch, labels = gen_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]], '->', labels[i,0], reverse_dictionary[labels[i,0]])

1276 alone -> 1869 apparently
1276 alone -> 36 home
1869 apparently -> 1276 alone
1869 apparently -> 71 st
71 st -> 256 louis
71 st -> 1869 apparently
256 louis -> 71 st
256 louis -> 791 plant


In [6]:
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.
num_sampled = 64      # Number of negative examples to sample.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. These 3 variables are used only for
# displaying model accuracy, they don't affect calculation.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

graph = tf.Graph()
with graph.as_default():
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size,1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    with tf.device('/cpu:0'):
        embeddings = tf.Variable(tf.random_uniform([volcabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

        #Construct the variables for the NCE loss
        nce_weights = tf.Variable(tf.truncated_normal([volcabulary_size, embedding_size], stddev=1.0/math.sqrt(embedding_size)))
        nce_biases=tf.Variable(tf.zeros([volcabulary_size]))

    #Average loss for the batch
    loss = tf.reduce_mean(
    tf.nn.nce_loss(
    weights=nce_weights,
    biases=nce_biases,
    labels=train_labels,
    inputs=embed,
    num_sampled=num_sampled,
    num_classes=volcabulary_size
    )
    )

    #Construct the SGD optimizer using a learning rate = 1.0
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

    # Add variable initializer.
    init = tf.global_variables_initializer()

# %% Begin Training
num_steps=50001
with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print('Initialized')

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = gen_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step ', step, ': ', average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = 'Nearest to %s:' % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = '%s %s,' % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  288.51953125
Nearest to world: atkins, bryant, leonard, holton, syringe, vermin, spurs, flamboyant,
Nearest to 11: citation, trampoline, cheeseburger, palestinians, gaunt, disclosure, branum, callenbach,
Nearest to percent: trolls, statuette, censorship, aquino, phylicia, hideaway, frou, 9463,
Nearest to two: pare, greenwood, andreasen, groping, crocodile, compromise, clowning, canoeing,
Nearest to school: housekeeping, bandmate, endocrine, donnelly, spasms, taster, basmati, minds,
Nearest to new: lundqvist, unrelated, 1936, methods, lei, deluca, discredit, coolant,
Nearest to 2: bravo, basing, officials, rincon, newsstands, superpower, kenjon, roc,
Nearest to money: tahoe, cochran, 1001, 1873, bolster, overseer, pained, palestinian,
Nearest to another: stewardship, powerless, overused, mde, pettit, mafia, lahood, gehrig,
Nearest to year: bimbo, warehouses, upswing, lament, tour, 482, linguistic, animas,
Nearest to part: manicured, lingers, 2100, 

Nearest to us: know, going, everybody, way, outfits, aqap, could, boynton,
Nearest to 7: 1, 8, 5, 2, 6, 4, 21, 9,
